In [1]:
import pandas as pd

# complete data-range
date0 = pd.to_datetime('2010-01-04')
date1 = pd.to_datetime('2024-11-30')
future = pd.to_datetime('2030-05-16')

def canonical_columns(df):
    """
    Converts the given DataFrame to a canonical format for Bitcoin price analysis.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame containing Bitcoin price data.
    
    Returns:
    - pd.DataFrame: A copy of the input DataFrame with the following modifications:
      - 'time' column converted to 'date' in datetime format, with time set to midnight.
      - Only 'date', 'time', and 'usd' (formerly 'open') columns are retained.
    """
    df = df.copy()
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['date'] = pd.to_datetime(df['date'].dt.date)
    df = df[['date', 'time', 'open']]
    df = df.rename(columns={'open': 'usd'})
    return df
 
def extend_dates(df, start_date, end_date):
    """
    Extends the given DataFrame to include future dates and fills missing values with the last known value.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame containing Bitcoin price data.
    - start_date (pd.Timestamp): The start date for the extension.
    - end_date (pd.Timestamp): The end date for the extension.
    
    Returns:
    - pd.DataFrame: A copy of the input DataFrame with the following modifications:
      - The 'date' column is extended to include future dates.
      - Missing values in the 'usd' column are filled with the last known value.
    """
    df = df.copy()
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    df = pd.merge(df, pd.DataFrame({'date': date_range}), on='date', how='outer')
    df.sort_values('date', inplace=True)

    # Use forward fill (ffill) to fill missing values with the last known value
    df['usd'] = df['usd'].ffill()
    return df

#btc = pd.read_csv('data/raw/BTCUSD.csv')
btc = pd.read_csv('data/BTCUSD_2024_11_21.csv')
btc = canonical_columns(btc)
btc = extend_dates(btc, date0, future)

# Set the condition to identify rows with dates larger than today
condition = btc['date'] > date1

# Erase data in the 'DataColumn' for dates larger than today
btc.loc[condition, 'usd'] = None  # Replace with None or any other value you prefer

display(btc)

,date,time,usd
0,2009-10-05,1.254701e+09,0.000959
1,2009-10-12,1.255306e+09,0.000965
2,2009-10-19,1.255910e+09,0.001067
3,2009-10-26,1.256515e+09,0.001163
4,2009-11-02,1.257120e+09,0.001205
...,...,...,...
7446,2030-05-12,NaN,NaN
7447,2030-05-13,NaN,NaN
7448,2030-05-14,NaN,NaN
7449,2030-05-15,NaN,NaN


In [2]:
import numpy as np

def _unix_to_btc_time(date):
    first = 1.2625632e+09
    delta = 1e7
    return date - first + delta

def _bitcoin_time(date):
    x = pd.to_datetime(date).copy()
    x = x.apply(lambda x: x.timestamp())
    return _unix_to_btc_time(x)

def _evaluate_fit(fit, date):
    # date -> btc-time
    x = pd.to_datetime(date)
    x = x.apply(lambda x: x.timestamp())
    x = _unix_to_btc_time(x)
    x = np.log(x)
    return np.exp(fit[0] * x + fit[1])

def btc_fit(dates, price):
    """
    Performs a logscale linear fit on the Bitcoin price data to predict future prices.

    Parameters:
    - dates (pd.Series): A series of dates for which the Bitcoin prices are known.
    - price (pd.Series): A series of Bitcoin prices corresponding to the dates.

    Returns:
    - pd.Series: A series of predicted Bitcoin prices based on the logscale linear fit.
    """
    t = _bitcoin_time(dates)
    t = np.log(t)
    y = np.log(price)
    btc_fit = np.polyfit(t, y, 1)
    return _evaluate_fit(btc_fit, btc['date'])

def _evaluate_fit_time(fit, price, time):
    # date -> btc-time
    return (np.exp((np.log(price) - fit[1]) / fit[0]) - time) / (24 * 60 * 60)

def btc_fit_time(dates, price):
    """
    This function calculates the time until the Bitcoin price reaches a certain value based on a logscale linear fit.
    
    Parameters:
    - dates (pd.Series): A series of dates for which the Bitcoin prices are known.
    - price (pd.Series): A series of Bitcoin prices corresponding to the dates.
    
    Returns:
    - pd.Series: A series of times until the Bitcoin price reaches the specified value.
    """
    t = _bitcoin_time(dates)
    t = np.log(t)
    y = np.log(price)
    btc_fit = np.polyfit(t, y, 1)
    return _evaluate_fit_time(btc_fit, btc['usd'], np.exp(t))

In [3]:
def log_fits(btc, last_included_date):
    # Create a copy of the original DataFrame
    df = btc.copy()
    # Limit the DataFrame to the specified date range
    df = df[df['date'] < pd.to_datetime(last_included_date)].copy()

    # Compute the main fit for the filtered range
    btc['fit'] = btc_fit(df['date'], df['usd'])
    btc['fit_time'] = btc_fit_time(df['date'], df['usd'])

    # Compute undervalued fit
    btc_undervalued = df[df['usd'] < btc.loc[df.index, 'fit']]  # Mask only for relevant indices
    btc['undervalued'] = btc_fit(btc_undervalued['date'], btc_undervalued['usd'])

    # Compute overvalued fit
    btc_overvalued = df[df['usd'] > btc.loc[df.index, 'fit']]  # Mask only for relevant indices
    btc['overvalued'] = btc_fit(btc_overvalued['date'], btc_overvalued['usd'])

    # Compute bubble fit
    btc_bubble = df[df['usd'] > btc.loc[df.index, 'overvalued']]  # Mask only for relevant indices
    btc['bubble'] = btc_fit(btc_bubble['date'], btc_bubble['usd'])

    # Compute top fit
    btc_top = df[df['usd'] > btc.loc[df.index, 'bubble']]  # Mask only for relevant indices
    btc['top'] = btc_fit(btc_top['date'], btc_top['usd'])

    return btc

today = '2024-11-30'
btc = log_fits(btc, last_included_date='2024-11-30')
display(btc)

,date,time,usd,fit,fit_time,undervalued,overvalued,bubble,top
0,2009-10-05,1.254701e+09,0.000959,1.668473e-07,118.511728,4.831624e-08,9.369211e-07,0.000009,0.000035
1,2009-10-12,1.255306e+09,0.000965,5.697244e-07,111.689668,1.715139e-07,3.016340e-06,0.000026,0.000096
2,2009-10-19,1.255910e+09,0.001067,1.521525e-06,107.646019,4.724982e-07,7.684984e-06,0.000060,0.000217
3,2009-10-26,1.256515e+09,0.001163,3.450151e-06,103.229623,1.099511e-06,1.675547e-05,0.000123,0.000426
4,2009-11-02,1.257120e+09,0.001205,6.960910e-06,97.313783,2.268109e-06,3.268671e-05,0.000228,0.000761
...,...,...,...,...,...,...,...,...,...
7446,2030-05-12,NaN,NaN,2.940202e+05,NaN,2.076274e+05,4.272591e+05,439524.641111,458118.618330
7447,2030-05-13,NaN,NaN,2.942122e+05,NaN,2.077673e+05,4.275248e+05,439775.491159,458365.825402
7448,2030-05-14,NaN,NaN,2.944044e+05,NaN,2.079073e+05,4.277906e+05,440026.451122,458613.133102
7449,2030-05-15,NaN,NaN,2.945966e+05,NaN,2.080473e+05,4.280565e+05,440277.521034,458860.541457


In [4]:
# Risk
v0 = np.log(btc['undervalued'])
v1 = np.log(btc['top'])
btc['risk'] = (np.log(btc['usd']) - v0) / (v1 - v0)

v0 = btc['undervalued']
v1 = btc['top']
btc['lin_risk'] = (btc['usd'] - v0) / (v1 - v0)
display(btc[['risk', 'lin_risk']])

,risk,lin_risk
0,1.503314,27.505438
1,1.364044,10.035129
2,1.260002,4.929132
3,1.168314,2.731497
4,1.078922,1.584273
...,...,...
7446,NaN,NaN
7447,NaN,NaN
7448,NaN,NaN
7449,NaN,NaN


In [5]:
from bitcoin_risk.plotter import bitcoin_risk
fig = bitcoin_risk(btc)
fig.write_html('docs/risk.html')
fig.show()

In [6]:
from bitcoin_risk.plotter import bitcoin_plot
fig = bitcoin_plot(btc)
fig.write_html('docs/btc.html', config={'displaylogo': False})
fig.write_image('docs/btc.png')
fig.show()

In [7]:
import plotly.express as px
fig = px.line(btc, x='date', y=['fit_time'],
              title='Bitcoin time risk - days until current price equals fair value',
              labels={'date': 'Date', 'open-price': 'Open Price', 
                      'fit': 'Fit', 
                      'undervalued': 'Undervalued',
                      'overvalued': 'Overvalued',
                      'top': 'Bubble Top'})
fig.update_traces(mode='lines', hovertemplate='%{y}')
fig.update_layout(
            shapes=[
            # Green (low risk)
            dict(
                type="rect",
                xref="paper", yref="y",
                x0=0, x1=1, y0=-750, y1=0,  # Adjust y0 and y1 for the "low risk" range
                fillcolor="rgba(0, 255, 0, 0.2)",  # Green with transparency
                layer="below", line_width=0
            ),
            # Orange (medium risk)
            dict(
                type="rect",
                xref="paper", yref="y",
                x0=0, x1=1, y0=0, y1=750,  # Adjust y0 and y1 for the "medium risk" range
                fillcolor="rgba(255, 165, 0, 0.2)",  # Orange with transparency
                layer="below", line_width=0
            ),
            # Red (high risk)
            dict(
                type="rect",
                xref="paper", yref="y",
                x0=0, x1=1, y0=750, y1=1300,  # Adjust y0 and y1 for the "high risk" range
                fillcolor="rgba(255, 0, 0, 0.2)",  # Red with transparency
                layer="below", line_width=0
            )])
fig.write_html('docs/timerisk.html', config={'displaylogo': False})

In [8]:
from bitcoin_risk.plotter import bitcoin_plot_time_risk
btc['risk'] = btc['fit_time']
fig = bitcoin_plot_time_risk(btc)
fig.write_html('docs/btc_time.html', config={'displaylogo': False})
fig.write_image('docs/btc.png')
fig.show()